In [1]:
import sys, platform, os
import numpy as np
import camb
import pandas as pd
import healpy as hp
from camb import model, initialpower
import useful_functions as uf
import pysm3
from fgbuster import (CMB, Dust, Synchrotron, basic_comp_sep,get_observation, get_instrument, get_sky)
from fgbuster.visualization import corner_norm
%load_ext autoreload
%autoreload 2
%matplotlib inline
import pylab
pylab.rcParams['figure.figsize'] = 12, 16

nside_patch = 16
nside=16
nside_compsep=nside_patch

seed_test=70
np.random.seed(seed_test)
#n_freq=len(sensitivities)
n_pix=hp.nside2npix(nside)

instrument = get_instrument('LiteBIRD')
kind_of_map='d0s0'
cmb='c1'
n_maps=1
pol=2

#sky_vary = get_sky(NSIDE_PATCH, kind_of_map)
sky_vary = get_sky(nside, kind_of_map)
cmb_sky=get_sky(nside, cmb)

'''for spectral_param in [sky_vary.components[0].mbb_index,
                       sky_vary.components[0].mbb_temperature,
                       sky_vary.components[1].pl_index]:
    spectral_param[:] = hp.ud_grade(hp.ud_grade(spectral_param.value, nside_patch),
                                    nside) * spectral_param.unit
    print(spectral_param.unit)'''

unit=-3 #uk->mk 
foreground_maps=np.zeros((n_maps,15,3,n_pix))
cmb_freq = get_observation(instrument, cmb_sky, noise=False, nside=nside,unit='mK_CMB')
for i in range(n_maps):
    sync_freq_maps = get_observation(instrument, sky_vary, noise=True, nside=nside,unit='mK_CMB') #, unit='uK_CMB')
    foreground_maps[i] = sync_freq_maps[:]+cmb_freq
#print(cmb_freq)

/home/amorelli/fgbuster/fgbuster/observation_helpers.py:97: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  return df.dropna(1, 'all')


In [3]:
inputs = [sky_vary.components[0].mbb_index,
          sky_vary.components[0].mbb_temperature.value,
          sky_vary.components[1].pl_index]
print(inputs)

[<Quantity 1.54>, 20.0, <Quantity -3.>]


In [11]:
components = [CMB(), Dust(150.), Synchrotron(20.)]
result=np.zeros((n_maps,n_pix,2))
for i in range(n_maps):
    compsep_tot=basic_comp_sep(components, instrument, foreground_maps[i,:,:1],nside=nside_compsep) #nside=0 is default
    compsep=compsep_tot.s[0]
    print(compsep.shape)
    #for p in range(pol):
        #result[i,:,p]=compsep[p+1]-cmb_freq[0,p+1,:]

(1, 3072)


In [12]:
outputs = [np.mean(compsep_tot.x[i]) for i in range(len(compsep_tot.x))]
print(outputs)

[1.540044310378392, 19.999264809311637, -3.000778092465398]


In [10]:
print((np.asarray(outputs)-np.asarray([1.540035111817395, 19.999755986910053, -3.0007962980019314]))/outputs)

[-3.26035943e-04  3.13453148e-05  3.26261268e-04]
